In [1]:
import requests
from nltk.corpus import wordnet
import urllib.request as urllib2
import urllib
import json
import gzip
from io import StringIO


class Dictionary:
    """
    A class used to access various online dictionaries and get synonyms, antonyms,
    definitions, and examples of a given word. This Class is for German Dictionaries.

    Args:
    dictionary_id (str): the id of the dictionary to use (one of 'dictionaryapi',
                         'oxford', 'wordnik', or 'wordnet')
    api_key (str, optional): the api key to use for the chosen dictionary
    source_lang (str, optional): the source language to use for the chosen dictionary

    Attributes:
    dictionary_id (str): the id of the dictionary being used
    api_key (str): the api key being used for the chosen dictionary (None if not required)
    source_lang (str): the source language being used for the chosen dictionary
    urls (dict): a dictionary mapping url types to urls for the chosen dictionary

    Methods:
    _get_urls(): get the urls for the chosen dictionary
    _make_request(url): make a request to the chosen dictionary
    get_synonyms(word): get the synonyms for a given word from the chosen dictionary
    get_antonyms(word): get the antonyms for a given word from the chosen dictionary
    get_definitions(word): get the definitions for a given word from the chosen dictionary
    get_examples(word): get the examples for a given word from the chosen dictionary
    """
    def __init__(self, dictionary_id, api_key=None, source_lang='DE'):
        """
        Initialize the Dictionary object with the given dictionary_id, api_key (if required),
        and source_lang (if applicable).
        """
        self.dictionary_id = dictionary_id
        self.api_key = api_key
        self.source_lang = source_lang
        self.urls = self._get_urls()
        self.ids = []

    def _get_id(self,word):
        """
        Get the IDs from the Babelnet for a given word
        """

        ids = []
        url = self.urls['ids'].format(word=word, api_key=self.api_key, source_lang=self.source_lang)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            for result in data:
                ids.append(result['id'])
            return ids
        else:
            return None

    def _get_urls(self):
        """
        Get the urls for the chosen dictionary based on its dictionary_id.
        """
        urls = {}

        if self.dictionary_id == "babel_ger":
            urls['ids'] = 'https://babelnet.io/v8/getSynsetIds?lemma={word}&searchLang={source_lang}&key={api_key}'
            urls['definitions'] = "https://babelnet.io/v8/getSynset?id={id}&targetLang={source_lang}&key={api_key}"
            urls['examples'] = "https://babelnet.io/v8/getSynset?id={id}&targetLang={source_lang}&key={api_key}"
        else:
            raise ValueError(f"Dictionary '{self.dictionary_id}' not supported")
        return urls

    def _make_request(self, url):
        """
        Make a request to the chosen dictionary with the given url.
        """
        if self.dictionary_id == 'wordnet':
            params = {'s': url}
            response = requests.get(self.urls['synonyms'], params=params)
            if response.status_code == 200:
                return response

            else:
                return None
        else:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response.json()
            else:
                return None


    def get_examples(self, id):
        """
        Get the definitions for a given id from the chosen dictionary.
        """
        all_examples = []
        url = self.urls['examples'].format(id =id, api_key=self.api_key, source_lang=self.source_lang)
        response = self._make_request(url)
        if response is None:
            return []
        if self.dictionary_id == "babel_ger":
            if len(response["glosses"]) > 0 :
                all_examples.append(response['glosses'][0]["gloss"])
        return all_examples




    def get_definitions(self, id):
        """
        Get the examples for a given word from the chosen dictionary.
        """

        all_definitions = []
        url = self.urls['definitions'].format(id =id, api_key=self.api_key, source_lang=self.source_lang)
        response = self._make_request(url)
        if response is None:
            return []
        if self.dictionary_id == "babel_ger":
            return response
        #     all_examples.append(response['glosses'][0]["gloss"])
        # return all_examples

        
        return list(all_definitions)
    
    def get_synonyms(self, word):
        """
        Get the synonyms for a given word from the chosen dictionary.
        """
        if self.dictionary_id == 'wordnet':
            synonyms = []
            for synset in wordnet.synsets(word):
                for lemma in synset.lemmas():
                    synonym = lemma.name().replace('_', ' ')
                    if synonym != word:
                        synonyms.append(synonym)
            return synonyms
        url = self.urls['synonyms'].format(word=word, api_key=self.api_key, source_lang=self.source_lang)
        response = self._make_request(url)
        if response:
                if self.dictionary_id == 'dictionaryapi':
                    synonyms = response[0].get('meta', {}).get('syns', [])[0]
                elif self.dictionary_id == 'oxford':
                    synonyms = response.get('results', [])[0].get('lexicalEntries', [])[0].get('entries', [])[0].get('senses', [])[0].get('synonyms', [])
                    synonyms = [synonym.get('text') for synonym in synonyms]
                elif self.dictionary_id == 'wordnik':
                    synonyms = response[0].get('words', [])
                return synonyms
        else:
            return None
    
    def get_antonyms(self, word):
        """
        Get the antonyms for a given word from the chosen dictionary.
        """
        if self.dictionary_id == 'wordnet':
                antonyms = []
                for synset in wordnet.synsets(word):
                    for lemma in synset.lemmas():
                        if lemma.antonyms():
                            antonyms.extend([antonym.name() for antonym in lemma.antonyms()])
                if antonyms:
                    return antonyms
        # url = self.urls['antonyms'].format(word=word, api_key=self.api_key, source_lang=self.source_lang)
        response = None
        if response:
            if self.dictionary_id == 'dictionaryapi':
                antonyms = response[0].get('meta', {}).get('ants', [])
                if antonyms:
                    return antonyms[0]
            elif self.dictionary_id == 'oxford':
                antonyms = []
                results = response.get('results', [])
                if results:
                    lexical_entries = results[0].get('lexicalEntries', [])
                    for entry in lexical_entries:
                        entries = entry.get('entries', [])
                        for entry in entries:
                            senses = entry.get('senses', [])
                            for sense in senses:
                                antonym_entries = sense.get('antonyms', [])
                                for antonym_entry in antonym_entries:
                                    antonyms.append(antonym_entry.get('text'))
                if antonyms:
                    return antonyms
            elif self.dictionary_id == 'wordnik':
                antonyms = []
                for result in response:
                    if result.get('relationshipType') == 'antonym':
                        words = result.get('words', [])
                        antonyms.extend(words)
                if antonyms:
                    return antonyms
        return None

In [2]:
dicti = Dictionary(dictionary_id="babel_ger", api_key="926b40cb-bd14-44ba-9ea3-788fe18ba5dc")
dicti.ids = dicti._get_id("Kanal")
for id in dicti.ids:
    print(dicti.get_examples(id))


['Der Kanal ist ein queerer Wagenplatz in Berlin.']
['Eine Meerenge, auch Meeresstraße oder Kanal, regional auch Sund oder Belt genannt, ist eine Stelle eines Meeres, an der sich zwei Landmassen nahe kommen und so einen Engpass des Meeres bilden.']
[]
['Lüftungskanäle oder Lüftungsrohre sind ein wesentlicher Bestandteil von Lüftungsanlagen und dienen der Luftführung.']
[]
['Als Kanal bezeichnet man eine Wasserstraße mit künstlich hergestelltem Gewässerbett.']
[]
['Flächeneinheit']
['Gemälde von Alfred Sisley']
[]
['Ein Kanal ist in der Informationstheorie ein Konzept, um den Informationsverlust durch Störungen bei der Übertragung zu modellieren.']
[]
[]
['Lithops ist eine Gattung sukkulenter Pflanzen aus der Familie der Mittagsblumengewächse und gehört zur Unterfamilie Ruschioideae.']
['Film von M. Padmakumar (2015)']
[]
[]
[]


In [96]:
definition = dicti.get_definitions(dicti.ids[0])

In [97]:
definition

{'senses': [{'type': 'BabelSense',
   'properties': {'fullLemma': 'Kanal_(Wagenplatz)',
    'simpleLemma': 'kanal',
    'lemma': {'lemma': 'Kanal', 'type': 'HIGH_QUALITY'},
    'source': 'WIKI',
    'senseKey': '1282836',
    'frequency': 0,
    'language': 'DE',
    'pos': 'NOUN',
    'synsetID': {'id': 'bn:17873360n', 'pos': 'NOUN', 'source': 'BABELNET'},
    'translationInfo': '',
    'pronunciations': {'audios': [], 'transcriptions': []},
    'bKeySense': False,
    'idSense': 235547885,
    'tags': {'class it.uniroma1.lcl.babelnet.data.LabelTag': [{'language': 'DE',
       'label': 'Wagenplatz'}]}}},
  {'type': 'BabelSense',
   'properties': {'fullLemma': 'Kanal',
    'simpleLemma': 'kanal',
    'lemma': {'lemma': 'Kanal', 'type': 'HIGH_QUALITY'},
    'source': 'WIKIDATA',
    'senseKey': 'Q2253992',
    'frequency': 0,
    'language': 'DE',
    'pos': 'NOUN',
    'synsetID': {'id': 'bn:17873360n', 'pos': 'NOUN', 'source': 'BABELNET'},
    'translationInfo': '',
    'pronunciation